<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Seq2SeqTranslation_With_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import numpy as np
import re
import tensorflow as tf
import os
import unicodedata
from tensorflow.keras.layers import *
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
if not os.path.exists('./datasets'):
  os.mkdir('./datasets')

The book, Deep Learning with tensorflow 2 and keras by PACKT, did not have an up to date version. This is my own way to extract the dataset

In [2]:
!wget  -P ./datasets https://www.manythings.org/anki/spa-eng.zip
!unzip ./datasets/spa-eng.zip -d ./datasets

--2021-02-13 18:23:17--  https://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3036::ac43:adc6, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4934182 (4.7M) [application/zip]
Saving to: ‘./datasets/spa-eng.zip’

spa-eng.zip         100%[===================>]   4.71M  15.6MB/s    in 0.3s    

2021-02-13 18:23:17 (15.6 MB/s) - ‘./datasets/spa-eng.zip’ saved [4934182/4934182]

Archive:  ./datasets/spa-eng.zip
  inflating: ./datasets/_about.txt   
  inflating: ./datasets/spa.txt      


In [3]:
def preprocess_sentence(sent):
  sent = ''.join([c for c in unicodedata.normalize('NFD', sent)])
  sent = re.sub(r'([!.?])', r' \1', sent)
  sent = re.sub(r'[^a-zA-Z!.?]+', r' ', sent)
  sent = re.sub(r'\s+', r' ', sent)
  sent = sent.lower()
  return sent

In [4]:
def download_and_read(num_pairs=50000):
    en_sents, sp_sents_in, sp_sents_out = [], [], []
    local_file = os.path.join("datasets", "spa.txt")
    with open(local_file, "r") as fin:
        for i, line in enumerate(fin):
            en_sent, sp_sent = line.strip().split('CC-BY')[0].strip().split('\t')
            en_sent = [w for w in preprocess_sentence(en_sent).split()]
            sp_sent = preprocess_sentence(sp_sent)
            sp_sent_in = [w for w in ("BOS " + sp_sent).split()]
            sp_sent_out = [w for w in (sp_sent + " EOS").split()]
            en_sents.append(en_sent)
            sp_sents_in.append(sp_sent_in)
            sp_sents_out.append(sp_sent_out)
            if i >= num_pairs-1:
              break
    return en_sents, sp_sents_in, sp_sents_out

In [5]:
sents_en, sents_sp_in, sents_sp_out = download_and_read()

tokenizer_en = tf.keras.preprocessing.text.Tokenizer(
    filters='',lower=False)
tokenizer_en.fit_on_texts(sents_en)
data_en = tokenizer_en.texts_to_sequences(sents_en)
data_en = tf.keras.preprocessing.sequence.pad_sequences(
    data_en, padding='post')

tokenizer_sp = tf.keras.preprocessing.text.Tokenizer(
    filters="", lower=False)
tokenizer_sp.fit_on_texts(sents_sp_in)
tokenizer_sp.fit_on_texts(sents_sp_out)
data_sp_in = tokenizer_sp.texts_to_sequences(sents_sp_in)
data_sp_in = tf.keras.preprocessing.sequence.pad_sequences(data_sp_in, padding="post")
data_sp_out = tokenizer_sp.texts_to_sequences(sents_sp_out)
data_sp_out = tf.keras.preprocessing.sequence.pad_sequences(data_sp_out, padding="post")

vocab_size_en = len(tokenizer_en.word_index)
vocab_size_sp = len(tokenizer_sp.word_index)

word2idx_en = tokenizer_en.word_index
idx2word_en = {v:k for k,v in word2idx_en.items()}

word2idx_sp = tokenizer_sp.word_index
idx2word_sp = {v:k for k,v in word2idx_sp.items()}

print('english vocabulary: ', str (vocab_size_en))
print('spanish vocabulary: ', str (vocab_size_sp))

maxlen_en = data_en.shape[1]
maxlen_sp = data_sp_out.shape[1]
print('The maximum english length is: {:d} '.format(maxlen_en) +   
  'and the maximum spanish length is: {:d}'.format(maxlen_sp))

english vocabulary:  6639
spanish vocabulary:  12364
The maximum english length is: 10 and the maximum spanish length is: 19


In [6]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices(
    (data_en, data_sp_in, data_sp_out))
dataset = dataset.shuffle(10000)
test_size = 5000 
test_dataset = dataset.take(test_size).batch(batch_size, drop_remainder=True)
train_dataset = dataset.skip(test_size).batch(batch_size, drop_remainder=True)

In [7]:
class BahdanauAttention(Layer):
  def __init__(self, num_units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(num_units)
    self.W2 = Dense(num_units)
    self.V = Dense(1)
  
  def call(self, query, values):
    
    query = tf.expand_dims(query, axis=1)
    distribution = self.V(tf.nn.tanh(self.W1(query)+ self.W2(values)))
    distribution = tf.nn.softmax(distribution)

    context = tf.reduce_sum(
        tf.linalg.matmul(
            tf.linalg.matrix_transpose(distribution),
            values
        ),axis=1
    )
    #context = tf.expand_dims(context,axis=1)
    return context, distribution

In [28]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, num_timesteps, 
            encoder_dim, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.encoder_dim = encoder_dim
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, embedding_dim, input_length=num_timesteps)
        self.rnn = tf.keras.layers.GRU(
            encoder_dim, return_sequences=True, return_state=True)

    def call(self, x, state):
        x = self.embedding(x)
        x, state = self.rnn(x, initial_state=state)
        return x, state

    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.encoder_dim))

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, decoder_dim, **kwargs):
    super(Decoder, self).__init__(**kwargs)
    self.decoder_dim = decoder_dim
    self.attention = BahdanauAttention(embedding_dim)
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.rnn = GRU(decoder_dim, return_sequences=True, return_state=True)
    self.Ws = Dense(vocab_size)

  def call(self, x, state, encoder_out):
    x = self.embedding(x)
    context, alignment = self.attention(state, encoder_out)
    x = tf.expand_dims(tf.concat([context, x], axis=-1),axis=1)
    x, state = self.rnn(x)
    x = self.Ws(x)
    return x, state, alignment

embedding_dim = 256
encoder_dim, decoder_dim = 256, 256

In [9]:
def build_graph():
  encoder = Encoder(vocab_size_en+1,embedding_dim, maxlen_en, encoder_dim)
  decoder = Decoder(vocab_size_sp+1, embedding_dim, decoder_dim)
  return encoder, decoder

In [10]:
  def loss_fn(ytrue, ypred):
    scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    mask = tf.math.logical_not(tf.math.equal(ytrue, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = scce(ytrue, ypred, sample_weight=mask)
    return loss

In [30]:
optimizer = tf.keras.optimizers.Adam()


@tf.function
def train_step(encoder, decoder, encoder_in, decoder_in, decoder_out, encoder_state):
  with tf.GradientTape() as tape:
    encoder_out, encoder_state = encoder(encoder_in, encoder_state)
    decoder_state = encoder_state
    loss = 0
    for t in range(decoder_out.shape[1]):
      decoder_in_t = decoder_in[:, t]
      decoder_pred_t, decoder_state, _ = decoder(decoder_in_t, decoder_state, encoder_out)
      loss += loss_fn(decoder_out[:, t], decoder_pred_t)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss / decoder_out.shape[1]

In [27]:
def predict(encoder, decoder, sents_en, data_en,
            sents_sp_out, word2idx_sp, idx2word_sp):
  random_id = np.random.choice(len(sents_en))
  print('input   : ', ' '.join(sents_en[random_id]))
  print('label   : ', ' '.join(sents_sp_out[random_id][0:-1]))
  encoder_in = tf.expand_dims(data_en[random_id],axis=0)
  decoder_out = tf.expand_dims(sents_sp_out[random_id], axis=0)
  encoder_state = encoder.init_state(1)
  encoder_out, encoder_state = encoder(encoder_in, encoder_state)
  decoder_state = encoder_state
  decoder_in = tf.constant([word2idx_sp['BOS']])
  pred_sent_sp = []
  while True:
    decoder_pred, decoder_state, _ = decoder(decoder_in, decoder_state, encoder_out)
    decoder_pred = tf.argmax(decoder_pred, axis=-1)
    pred_word = idx2word_sp[decoder_pred.numpy()[0][0]]
    pred_sent_sp.append(pred_word)
    if pred_word == 'EOS':
      break
    decoder_in = decoder_pred[0]
  print('predicted   : ', ' '.join(pred_sent_sp))

In [31]:
 with tf.device('gpu:0'):
  encoder, decoder = build_graph()
  if not os.path.exists('./checkpoints'):
    os.mkdir('./checkpoints')
  num_epochs = 50
  for e in range(1,num_epochs+1):
    encoder_state = encoder.init_state(batch_size)
    for batch, data in enumerate(train_dataset):
      encoder_in, decoder_in, decoder_out = data 
      loss = train_step(encoder, decoder, encoder_in, decoder_in, decoder_out, encoder_state)
    print('Epoch: {}/{}, Loss: {:.4f}'.format(e,num_epochs, loss.numpy()))
    if e % 5 == 0:
      print('PREDICTING TEXT \n')
      predict(encoder, decoder, sents_en, data_en, sents_sp_out,word2idx_sp, idx2word_sp)
      print('\n')
      if e % 2 == 0:
        print('SAVING CHECKPOINT {} ...'.format(str (e // 10)))
        encoder.save_weights('./checkpoints/encoder_ckpt_{}.h5'.format(str (e // 10)))
        decoder.save_weights('./checkpoints/decoder_ckpt_{}.h5'.format(str (e // 10)))


Epoch: 1/50, Loss: 2.1898
Epoch: 2/50, Loss: 2.1438
Epoch: 3/50, Loss: 1.2886
Epoch: 4/50, Loss: 0.9837
Epoch: 5/50, Loss: 0.8524
PREDICTING TEXT 

input   :  i don t quite follow you .
label   :  no veo a do nde vas .
predicted   :  no te he dicho . EOS


Epoch: 6/50, Loss: 0.5914
Epoch: 7/50, Loss: 0.4836
Epoch: 8/50, Loss: 0.3837
Epoch: 9/50, Loss: 0.3739
Epoch: 10/50, Loss: 0.3307
PREDICTING TEXT 

input   :  be kind .
label   :  sean gentiles .
predicted   :  se dico . EOS


SAVING CHECKPOINT 1 ...
Epoch: 11/50, Loss: 0.3014
Epoch: 12/50, Loss: 0.2539
Epoch: 13/50, Loss: 0.2039
Epoch: 14/50, Loss: 0.1743
Epoch: 15/50, Loss: 0.1419
PREDICTING TEXT 

input   :  i will help you all i can .
label   :  te ayudare en todo lo que pueda .
predicted   :  te ayudare en todo lo que pueda . EOS


Epoch: 16/50, Loss: 0.1659
Epoch: 17/50, Loss: 0.1370
Epoch: 18/50, Loss: 0.1019
Epoch: 19/50, Loss: 0.1091
Epoch: 20/50, Loss: 0.0971
PREDICTING TEXT 

input   :  this is a triangle .
label   :  est